# ML-Leaks Paper implementation:

>This is an implementation of adversary 1 scenario of the paper (ML-Leaks)[https://arxiv.org/pdf/1806.01246.pdf]

## I. General Idea of the paper:

As machine learning as a service (Mlaas) is getting widely used, privacy issues in this setting are also getting a lot of attention. One of the most famous attacks that a depolyed machine learning model can be victim of is "the membership inference attack". This attack allows an adversary to know if a particular data point was used to trained a given model. In case of a model trained on highly private data, this attack can be a real danger.

Early demos of memebership infrence attacks assumed the following :

- Knowledge of the target model architecture.

- Using multiple shadow models.

- Having shadow models trained on datasets that are from the same distribution of the dataset used to train the target model itself.

Despite the high performance of such attacks, the existence of these assumptions made it unrealistic to be deployed in real world. In this paper, the authors relaxed these assumptions and suggested some effective defense strategies , namely : Dropout and model stacking.

The authors designed 3 adversary strategies :

- Adversary 1 : Used only one shadow model + no knowledge of the classification algorithm used. ==> The usage of only one shadow model resulted in a similar performance compared to using many shadow models. This can make the attacks much more computationally efficient.

- Adversary 2 : The data sued to train the shadow model isn't necessarily from the same distribution of the data used to train the target model ==> This makes the attack more powerful and realistic.

- Adversary 3 : No shadow models are used + an attack in an unsupervised attack ==> despite the drop of the performance for this attck it is still effective.

> In this notebook we are implementing the Adversary 1 and testing on Mnist dataset and Cifar10.

### Table of Content:
<ol>
    <li> <a href="#case1">Case 1 : using the same architecture for the target and the shadow</a></li>
        <ol>
            <li><a href="#mnist1">Mnist study case</a></li>
            <li><a href="#dataloaders">Load data</a></li>
            <li><a href="#training">Training models</a></li>
            <li><a href="#eval">Some Extra evaluation results</a></li>
        </ol>
     </li>
    <li> <a href="#mnist">Cifar10 dataset case</a>
        <ol>
            <li><a href="#dataloaders2">Load data</a></li>
            <li><a href="#training2">Training models</a></li>
            <li><a href="#eval2">Some Extra evaluation results</a></li>
        </ol>
     </li>
</ol>


> Important note : The number of epochs displayed when training may not be the exact number of training epochs used for the final model. By the time I was running tests , it happened when I needed to add epochs and run again the training cell ( to improuve the accuracy/loss) 

In [2]:
# Importing necessary packages 
from models.models import *
import dataloaders as loader
import training_utils as TU
from torch.optim import *
from datasets.data_utils import *
from datasets.case2_data_utils import *
from datasets.attack_dataset import *  
import pandas as pd 
from sklearn.metrics import classification_report
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle


## II. Case 1 : Same model architecture for the shadow and the target <a id="case1"></a>

### 1.1 Mnist classifier: <a id ="mnist1"></a>

#### 1.1.1 Load data :

In [2]:
loaders = loader.get_dataloaders(batch_size=128,dataset_name="mnist")

In [3]:
loaders.keys()

dict_keys(['DShadow_train', 'DShadow_out', 'target_train', 'target_eval', 'test_loader'])

#### 1.1.2 train and evaluate the target model:

In [3]:
target_model = Mnist_classifier()

In [4]:
train_dataloader = loaders['target_train']
optimizer = Adam(target_model.parameters(), lr=0.0001)
epochs = 30
checkpoint_path = "./checkpoints/case1/mnistTarget.pth"
target_train_loss_scores = TU.train(train_dataloader=train_dataloader, 
                                    optimizer=optimizer,
                                    model=target_model,
                                    n_epochs=epochs,
                                    model_path=checkpoint_path)

Epoch 1 - Training loss: 1.7336031607652114
Epoch 2 - Training loss: 0.6991646052417109
Epoch 3 - Training loss: 0.43768555153224425
Epoch 4 - Training loss: 0.3306552916765213
Epoch 5 - Training loss: 0.2674894731933788
Epoch 6 - Training loss: 0.22669988011909745
Epoch 7 - Training loss: 0.19536167168516225
Epoch 8 - Training loss: 0.17107124607694352
Epoch 9 - Training loss: 0.15237095478480145
Epoch 10 - Training loss: 0.13591645995817953
Epoch 11 - Training loss: 0.12330653928851677
Epoch 12 - Training loss: 0.1126803884251138
Epoch 13 - Training loss: 0.10286486606602951
Epoch 14 - Training loss: 0.09479775257661181
Epoch 15 - Training loss: 0.08915792882316194
Epoch 16 - Training loss: 0.08216708196106098
Epoch 17 - Training loss: 0.07772603141680612
Epoch 18 - Training loss: 0.0728833022195909
Epoch 19 - Training loss: 0.06714723495987512
Epoch 20 - Training loss: 0.0630572308012742
Epoch 21 - Training loss: 0.059463714375713114
Epoch 22 - Training loss: 0.055863706426600275
Ep

In [5]:
# eval
acc = TU.eval_model(target_model,loaders['test_loader'])


Average Val Loss: 0.0004, Val Accuracy: 9846/10000 (98.460%)



In [6]:
#Save results 
TU.save_training_loss(target_train_loss_scores,"case1/target_model_train_scores.csv")

#### 1.1.2 train and evaluate the shadow model:

In [7]:
shadow_model = Mnist_classifier()

In [8]:
train_dataloader = loaders['DShadow_train']
optimizer = Adam(shadow_model.parameters(), lr=0.0001)
epochs = 30
checkpoint_path = "./checkpoints/case1/mnistShadow.pth"
shadow_train_loss_scores = TU.train(train_dataloader=train_dataloader, 
                                    optimizer=optimizer,
                                    model=shadow_model,
                                    n_epochs=epochs,
                                    model_path=checkpoint_path)

Epoch 1 - Training loss: 1.650734110403869
Epoch 2 - Training loss: 0.6618791730222056
Epoch 3 - Training loss: 0.4201601612365852
Epoch 4 - Training loss: 0.31804574590365764
Epoch 5 - Training loss: 0.25947067652971056
Epoch 6 - Training loss: 0.22189391688522647
Epoch 7 - Training loss: 0.19272407453696608
Epoch 8 - Training loss: 0.16948357173952006
Epoch 9 - Training loss: 0.15162894909538455
Epoch 10 - Training loss: 0.13634572556968463
Epoch 11 - Training loss: 0.12467115845973209
Epoch 12 - Training loss: 0.1144538798953517
Epoch 13 - Training loss: 0.1062625774766429
Epoch 14 - Training loss: 0.09781695144661402
Epoch 15 - Training loss: 0.09179458075787052
Epoch 16 - Training loss: 0.08455126115389294
Epoch 17 - Training loss: 0.07867810913061692
Epoch 18 - Training loss: 0.07313994658416358
Epoch 19 - Training loss: 0.06843136344105005
Epoch 20 - Training loss: 0.06478635789984363
Epoch 21 - Training loss: 0.061030569454749765
Epoch 22 - Training loss: 0.05771656714821771
Ep

In [9]:
acc_shadow = TU.eval_model(shadow_model,loaders['test_loader'])


Average Val Loss: 0.0004, Val Accuracy: 9825/10000 (98.250%)



In [10]:
#Save results 
TU.save_training_loss(shadow_train_loss_scores,"case1/shadow_model_train_scores.csv")

#### 1.1.3 train and evaluate the attack model:

##### 1.1.3.1 Create dataset of the attack model:

In [3]:
# The dataset is already created and saved in data/data/attack_dataset.csv so there s no need 
# to run this cell 
train_file_path='data/case1/mnist/attack_train_dataset.csv'
test_file_path='data/case1/mnist/attack_test_dataset.csv'
# In short the fowllong means that:
#  Attack_train = top3(shadow(D_shadow)) = top3(shadow(D_shadwo_train + D_shadow_out_shadow)) 
result = get_attack_trainset(shadow_model=shadow_model,
                             shadow_train_loader=loaders['DShadow_train'], 
                             shadow_out_loader=loaders['DShadow_out'],
                             file_path=train_file_path)
result = get_attack_trainset(shadow_model=shadow_model,
                             shadow_train_loader=loaders['target_train'], 
                             shadow_out_loader=loaders['target_eval'],
                             file_path=test_file_path)

In [2]:
train_file_path='data/case1/mnist/attack_train_dataset.csv'
test_file_path='data/case1/mnist/attack_test_dataset.csv'
attack_loaders = loader.get_attack_train_test_loaders(dataset_train_path=train_file_path,
                                                      dataset_test_path=test_file_path,
                                                        batch_size=10)

##### 1.1.3.2 train and evaluate the attack model:

In [3]:
attack_model = Attack_classifier(3)

In [4]:
train_dataloader = attack_loaders['train_loader']


In [5]:
optimizer = Adam(attack_model.parameters(), lr=0.00001)

train_scores= TU.train(train_dataloader, 
                    optimizer,
                    model=attack_model,
                    n_epochs=20, 
                    model_path="./checkpoints/case1/mnist_attack_model.pth")

Epoch 1 - Training loss: 0.6961096758445104
Epoch 2 - Training loss: 0.6956390447219213
Epoch 3 - Training loss: 0.6959529554247856
Epoch 4 - Training loss: 0.6956756233970324
Epoch 5 - Training loss: 0.6961275424361228
Epoch 6 - Training loss: 0.6961167560418446
Epoch 7 - Training loss: 0.696594585955143
Epoch 8 - Training loss: 0.6959110643863679
Epoch 9 - Training loss: 0.6963876638611157
Epoch 10 - Training loss: 0.6960333439906439
Epoch 11 - Training loss: 0.6956823712587357
Epoch 12 - Training loss: 0.6963086466987928
Epoch 13 - Training loss: 0.6951838530699412
Epoch 14 - Training loss: 0.6957121727267901
Epoch 15 - Training loss: 0.6960745484630266
Epoch 16 - Training loss: 0.6964540767272314
Epoch 17 - Training loss: 0.6954973247647286
Epoch 18 - Training loss: 0.6959402126471201
Epoch 19 - Training loss: 0.6955236923098564
Epoch 20 - Training loss: 0.6957927646835645


In [7]:
acc_attack,precision_attack,recall_attack = TU.eval_model(attack_model,attack_loaders['test_loader'])


Average Val Loss: 0.0693, Val Accuracy: (0.499%) , precision:  0.4994, recall :  0.8363



In [8]:
#Save results 
TU.save_training_loss(train_scores,"case1/attack_model_train_scores.csv")

In [11]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
raw_data = pd.read_csv('data/attack_train_dataset.csv')
x = raw_data[['top1', 'top2', 'top3']]
y = raw_data['label']
test_data = pd.read_csv("data/attack_test_dataset.csv")
x_test = test_data[['top1', 'top2', 'top3']]
y_test = test_data['label']
model = LogisticRegression()
model.fit(x, y)
predictions = model.predict(x_test)
print(len(predictions))
l = classification_report(y_test, predictions)

30000


### 1.2 Cifar10 classifier :

#### 1.2.1 Load data :

In [28]:
cifar_loaders = loader.get_dataloaders(batch_size=10,dataset_name="cifar10")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
cifar_loaders.keys()

dict_keys(['DShadow_train', 'DShadow_out', 'target_train', 'target_eval', 'test_loader'])

#### 1.2.2 train and evaluate the target model:

In [3]:
cifar_target_model = Cifar10_classifier()

In [35]:
cifar_train_dataloader = cifar_loaders['target_train']
optimizer = Adam(cifar_target_model.parameters(), lr=0.0000001)
epochs = 20
checkpoint_path = "./checkpoints/case1/cifarTarget.pth"
target_train_loss_scores = TU.train(train_dataloader=cifar_train_dataloader, 
                                    optimizer=optimizer,
                                    model=cifar_target_model,
                
                                    n_epochs=epochs,
                                    model_path=checkpoint_path)

Epoch 1 - Training loss: 0.9930657721757888
Epoch 2 - Training loss: 0.9898408813476562
Epoch 3 - Training loss: 0.9950257426738739
Epoch 4 - Training loss: 0.9973889088630676
Epoch 5 - Training loss: 0.9918086815357208
Epoch 6 - Training loss: 0.9929179166316986
Epoch 7 - Training loss: 0.9986713723540306
Epoch 8 - Training loss: 0.9972829787969589
Epoch 9 - Training loss: 0.9918373112916946
Epoch 10 - Training loss: 0.9968669293642044
Epoch 11 - Training loss: 0.9972882260322571
Epoch 12 - Training loss: 0.9936566089391708
Epoch 13 - Training loss: 1.0040358982563018
Epoch 14 - Training loss: 0.9965535270810127
Epoch 15 - Training loss: 0.9984656764984131
Epoch 16 - Training loss: 0.9938789386034012
Epoch 17 - Training loss: 1.0027177766084672
Epoch 18 - Training loss: 0.9959812175512314
Epoch 19 - Training loss: 0.9889417807340622
Epoch 20 - Training loss: 0.9876851892709732


In [34]:
target_acc , target_precision , target_recall  = TU.eval_model(cifar_target_model,
                                                               cifar_loaders['test_loader'],attack=False)


Average Val Loss: 0.1111, Val Accuracy: (0.607%) , precision:  0.6065, recall :  0.6065



In [37]:
#Save results 
TU.save_training_loss(target_train_loss_scores,"case1/cifar_target_model_train_scores.csv")

#### 1.2.2 train and evaluate the shadow model:

In [40]:
cifar_shadow_model = Cifar10_classifier()

In [51]:
train_dataloader = cifar_loaders['DShadow_train']
optimizer = Adam(cifar_shadow_model.parameters(), lr=0.0000001)
epochs = 20
checkpoint_path = "./checkpoints/case1/cifar_Shadow.pth"
shadow_train_loss_scores = TU.train(train_dataloader=train_dataloader, 
                                    optimizer=optimizer,
                                    model=cifar_shadow_model,
                                    n_epochs=epochs,
                                    model_path=checkpoint_path)

Epoch 1 - Training loss: 1.2923674753904342
Epoch 2 - Training loss: 1.3029212895393372
Epoch 3 - Training loss: 1.2898649885177613
Epoch 4 - Training loss: 1.2910157915592193
Epoch 5 - Training loss: 1.291897402739525
Epoch 6 - Training loss: 1.286360319519043
Epoch 7 - Training loss: 1.2983051123857499
Epoch 8 - Training loss: 1.2907833490610123
Epoch 9 - Training loss: 1.3014600640296936
Epoch 10 - Training loss: 1.2954328127384185
Epoch 11 - Training loss: 1.294667185974121
Epoch 12 - Training loss: 1.2911283937215805
Epoch 13 - Training loss: 1.2865879675388336
Epoch 14 - Training loss: 1.298851662826538
Epoch 15 - Training loss: 1.2897569953441619
Epoch 16 - Training loss: 1.299311732339859
Epoch 17 - Training loss: 1.2858497423171997
Epoch 18 - Training loss: 1.2839252040863036
Epoch 19 - Training loss: 1.2928462571620942
Epoch 20 - Training loss: 1.2939718429803848


In [54]:
shadow_acc , shadow_precision , shadow_recall  = TU.eval_model(cifar_shadow_model,
                                                               cifar_loaders['test_loader'],attack=False)


Average Val Loss: 0.1112, Val Accuracy: (0.606%) , precision:  0.6057, recall :  0.6057



In [55]:
TU.save_training_loss(shadow_train_loss_scores,"case1/cifar_shadow_model_train_scores.csv")

#### 1.2.3 train and evaluate the attack model:

##### 1.1.3.1 Create dataset of the attack model:

In [56]:
# The dataset is already created and saved in data/data/attack_dataset.csv so there s no need 
# to run this cell 
train_file_path='data/case1/cifar10/attack_train_dataset.csv'
test_file_path='data/case1/cifar10/attack_test_dataset.csv'
# In short the fowllong means that:
#  Attack_train = top3(shadow(D_shadow)) = top3(shadow(D_shadwo_train + D_shadow_out_shadow)) 
result = get_attack_trainset(shadow_model=cifar_shadow_model,
                             shadow_train_loader=cifar_loaders['DShadow_train'], 
                             shadow_out_loader=cifar_loaders['DShadow_out'],
                             file_path=train_file_path)
result = get_attack_trainset(shadow_model=cifar_shadow_model,
                             shadow_train_loader=cifar_loaders['target_train'], 
                             shadow_out_loader=cifar_loaders['target_eval'],
                             file_path=test_file_path)

/home/soumia/cispa/ML-Leaks/datasets/data_utils.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [57]:
train_file_path='data/case1/cifar10/attack_train_dataset.csv'
test_file_path='data/case1/cifar10/attack_test_dataset.csv'
attack_loaders = loader.get_attack_train_test_loaders(dataset_train_path=train_file_path,
                                                      dataset_test_path=test_file_path,
                                                        batch_size=10)

##### 1.2.3.2 train and evaluate the attack model:

In [58]:
attack_model = Attack_classifier(3)

In [59]:
train_dataloader = attack_loaders['train_loader']

In [62]:
optimizer = Adam(attack_model.parameters(), lr=0.000001)

train_scores= TU.train(train_dataloader, 
                    optimizer,
                    model=attack_model,
                    n_epochs=20, 
                    model_path="./checkpoints/case1/cifar_attack_model.pth")

Epoch 1 - Training loss: 0.6950505030632019
Epoch 2 - Training loss: 0.6957265115261078
Epoch 3 - Training loss: 0.6949146528244019
Epoch 4 - Training loss: 0.695542797613144
Epoch 5 - Training loss: 0.6957440102815629
Epoch 6 - Training loss: 0.6952755299568176
Epoch 7 - Training loss: 0.6952428996801376
Epoch 8 - Training loss: 0.6954595716953278
Epoch 9 - Training loss: 0.6952077157020569
Epoch 10 - Training loss: 0.6956053576946258
Epoch 11 - Training loss: 0.6953654679775239
Epoch 12 - Training loss: 0.6953147011995315
Epoch 13 - Training loss: 0.6949078891038895
Epoch 14 - Training loss: 0.6951793418169021
Epoch 15 - Training loss: 0.6953741498470306
Epoch 16 - Training loss: 0.6959248410224914
Epoch 17 - Training loss: 0.6951125086069107
Epoch 18 - Training loss: 0.6948982631444931
Epoch 19 - Training loss: 0.695225207734108
Epoch 20 - Training loss: 0.6953412457942962


In [63]:
acc_attack,precision_attack,recall_attack = TU.eval_model(attack_model,attack_loaders['test_loader'])


Average Val Loss: 0.0693, Val Accuracy: (0.502%) , precision:  0.5022, recall :  0.5410



## II. Case 2 : Shadow model's architecture different than the target model's architecture aka Combining attack:

### 2.1 Mnist classifier :

In [64]:
# Since it's the same model I just load it again 
# This cell is not mandatory 
target_model_mnist2 = Mnist_classifier()
target_model_mnist2.load_state_dict(torch.load( "./checkpoints/case1/mnistTarget.pth"))

<All keys matched successfully>

#### 2.1.2 train and evaluate the shadow model:
Here the shadow model is a combined model of 3 models each is a submodel(Random forests , Logistic regression) 

In [98]:
# Load and split data for the sub classifiers 
data = pd.read_csv("./mnist csv/Mnist.csv")
x=train.iloc[:,1:]
y=train.iloc[:,0]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=0)
x_D_shadow , x_D_target,y_D_shadow , y_D_target  = train_test_split(x_train, y_train, test_size=0.5, random_state=0)
x_D_shadow_train, x_D_shadow_out, y_D_shadow_train, y_D_shadow_out  = train_test_split(x_D_shadow, y_D_shadow, test_size=0.5, random_state=0)
x_D_target_train, x_D_target_eval, y_D_target_train, y_D_target_eval = train_test_split(x_D_target, y_D_target, test_size=0.5, random_state=0)

##### 2.1.2.1 Sub shadow model 1 : Random Forests : 

In [111]:
rf = RandomForestClassifier(n_estimators =50, random_state = 42)
rf.fit(x_D_shadow , y_D_shadow )

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [112]:
#eval 
pred=rf.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       193
           1       0.98      0.97      0.98       237
           2       0.95      0.98      0.96       208
           3       0.94      0.94      0.94       215
           4       0.95      0.96      0.95       223
           5       0.97      0.94      0.96       197
           6       0.96      0.98      0.97       204
           7       0.98      0.97      0.98       224
           8       0.93      0.92      0.93       201
           9       0.94      0.91      0.92       198

    accuracy                           0.95      2100
   macro avg       0.95      0.95      0.95      2100
weighted avg       0.95      0.95      0.95      2100



In [125]:
#Save model 
filename = './checkpoints/case2/RandomForest_mnist.sav'
pickle.dump(rf, open(filename, 'wb'))

##### 2.1.2.2 Sub shadow model  : Logistic Regression  : 

In [104]:
lr = LogisticRegression()
lr.fit(x_D_shadow , y_D_shadow )

/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
pred2=np.floor(lr.predict(x_test))
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95       193
           1       0.97      0.98      0.98       237
           2       0.92      0.91      0.91       208
           3       0.89      0.89      0.89       215
           4       0.92      0.88      0.90       223
           5       0.86      0.87      0.87       197
           6       0.89      0.95      0.92       204
           7       0.92      0.96      0.94       224
           8       0.88      0.84      0.86       201
           9       0.88      0.86      0.87       198

    accuracy                           0.91      2100
   macro avg       0.91      0.91      0.91      2100
weighted avg       0.91      0.91      0.91      2100



In [124]:
#Save model 
filename = './checkpoints/case2/LogisticRegression_mnist.sav'
pickle.dump(lr, open(filename, 'wb'))

##### 2.1.2.3 Stacking submodels : 

In [114]:
from sklearn.ensemble import StackingClassifier
estimators = [
            ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
            ('lr', LogisticRegression())
]
clf = StackingClassifier(
        estimators=estimators)

In [115]:
 clf.fit(x_D_shadow , y_D_shadow)

/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [116]:
#eval 
pred3=clf.predict(x_test)
print(classification_report(y_test, pred3))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       193
           1       0.99      0.97      0.98       237
           2       0.96      0.98      0.97       208
           3       0.93      0.94      0.94       215
           4       0.96      0.94      0.95       223
           5       0.97      0.95      0.96       197
           6       0.94      0.99      0.96       204
           7       0.98      0.97      0.98       224
           8       0.93      0.93      0.93       201
           9       0.93      0.92      0.92       198

    accuracy                           0.95      2100
   macro avg       0.95      0.95      0.95      2100
weighted avg       0.95      0.95      0.95      2100



In [123]:
# Save the model 
filename = './checkpoints/case2/ensemble_mnist.sav'
pickle.dump(clf, open(filename, 'wb'))

##### 2.1.3 construct Attack data : <a id="something"></a>

In [420]:
def construct_dataset(x1, x2, clf, path_save):
    "assuming x1,y1 are Dtrain shadow and x2 , y2 are Dshadow out"
    df = pd.DataFrame()
    probas = clf.predict_proba(x1)
    sorted_probs = np.sort(probas, axis=1)
    
    for e in sorted_probs: 
        resu = e[-3:]
        df2 = pd.DataFrame({"top1": resu[0],
                            "top2":resu[1],
                            "top3": resu[2],
                            "label": 1
                            },index=[0])
        df = df.append(df2)
     
    probas = clf.predict_proba(x2)
    sorted_probs = np.sort(probas, axis=1)
    for e in sorted_probs: 
        resu = e[-3:]
        df3 = pd.DataFrame({"top1": resu[0],
                            "top2":resu[1],
                            "top3": resu[2],
                            "label": 0
                            },index=[0])
        df = df.append(df)
    df.to_csv(path_save,index=False)
    return df
    
    

In [182]:
# This cell has already been processed and data are saved 
# There is no need to re-run it again
attack_train_save = "data/case2/mnist/attack_train_dataset.csv"
attack_test_save = "data/case2/mnist/attack_test_dataset.csv"

df = construct_dataset(x1=x_D_shadow_train,
                       x2=x_D_shadow_out,
                       clf=clf, 
                       path_save=attack_train_save)

df = construct_dataset(x1=x_D_target_train,
                       x2=x_D_target_eval,
                       clf=clf, 
                       path_save=attack_test_save)

In [183]:
# Load data 
train_file_path="data/case2/mnist/attack_train_dataset.csv"
test_file_path="data/case2/mnist/attack_test_dataset.csv"
attack_loaders = loader.get_attack_train_test_loaders(dataset_train_path=train_file_path,
                                                      dataset_test_path=test_file_path,
                                                        batch_size=10)

##### 2.1.4 train attack model and evaluate it : 

In [184]:
attack_model = Attack_classifier(3)

In [185]:
train_dataloader = attack_loaders['train_loader']

In [188]:
optimizer = Adam(attack_model.parameters(), lr=0.0001)

train_scores= TU.train(train_dataloader, 
                    optimizer,
                    model=attack_model,
                    n_epochs=20, 
                    model_path="./checkpoints/case2/mnist_attack_model.pth")

Epoch 1 - Training loss: 0.6931727687517802
Epoch 2 - Training loss: 0.6933887512164009
Epoch 3 - Training loss: 0.6932129659449546
Epoch 4 - Training loss: 0.6934609806627259
Epoch 5 - Training loss: 0.6932945775507685
Epoch 6 - Training loss: 0.6933378587390546
Epoch 7 - Training loss: 0.6933012390794013
Epoch 8 - Training loss: 0.6932324128640923
Epoch 9 - Training loss: 0.6932284658415275
Epoch 10 - Training loss: 0.6933061804986538
Epoch 11 - Training loss: 0.6932561711858687
Epoch 12 - Training loss: 0.6931575557641816
Epoch 13 - Training loss: 0.6931611282484872
Epoch 14 - Training loss: 0.6932790222622099
Epoch 15 - Training loss: 0.6932108251672042
Epoch 16 - Training loss: 0.6932844067277167
Epoch 17 - Training loss: 0.6931624237756083
Epoch 18 - Training loss: 0.6931881216235627
Epoch 19 - Training loss: 0.6931945237898289
Epoch 20 - Training loss: 0.6932244052863061


In [189]:
#eval 
acc_attack,precision_attack,recall_attack = TU.eval_model(attack_model,attack_loaders['test_loader'])


Average Val Loss: 0.0693, Val Accuracy: (0.500%) , precision:  0.4998, recall :  0.9398



### 2.2 Cifar 10 classifier :

#### 2.2.1 Train target model : 

In [213]:
# Since it's the same model I just load it again 
# This cell is not mandatory 
target_model_cifar2 = Cifar10_classifier()
target_model_cifar2.load_state_dict(torch.load( "./checkpoints/case1/cifarTarget.pth"))

#### 2.2.2 Train and test shadow model:
Here the shadow model is a combined model of 3 models each is a submodel(Random forests , Logistic regression) 

In [3]:
from keras.datasets import cifar10
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [17]:
(x_train, y_train), (x_test, y_test)= cifar10.load_data()

In [5]:
y_train = label_binarize(y_train, classes=[0,1,2,3,4,5,6,7,8,9])
y_test = label_binarize(y_test, classes=[0,1,2,3,4,5,6,7,8,9])

In [18]:
x_D_shadow , x_D_target,y_D_shadow , y_D_target  = train_test_split(x_train, y_train, test_size=0.5, random_state=0)
x_D_shadow_train, x_D_shadow_out, y_D_shadow_train, y_D_shadow_out  = train_test_split(x_D_shadow, y_D_shadow, test_size=0.5, random_state=0)
x_D_target_train, x_D_target_eval, y_D_target_train, y_D_target_eval = train_test_split(x_D_target, y_D_target, test_size=0.5, random_state=0)

In [19]:
x_D_shadow_train = x_D_shadow_train.reshape(len(x_D_shadow_train), 3*32*32)
x_D_shadow_out = x_D_shadow_out.reshape(len(x_D_shadow_out), 3*32*32)
x_D_target_train = x_D_target_train.reshape(len(x_D_target_train), 3*32*32)
x_D_target_eval= x_D_target_eval.reshape(len(x_D_target_eval), 3*32*32)
x_test = x_test.reshape(len(x_test), 3*32*32)

##### 2.2.2.1 Sub shadow model  : Logistic Regression  : 

In [17]:
lr_one_rest = OneVsRestClassifier(LogisticRegression(random_state=42, multi_class='multinomial'))
lr_one_rest.fit(x_D_shadow_train, y_D_shadow_train)

/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='multinomial',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [8]:
#test 
pred4=lr_one_rest.predict(x_test)
print(classification_report(y_test, pred4))

              precision    recall  f1-score   support

           0       0.37      0.20      0.26      1000
           1       0.55      0.29      0.38      1000
           2       0.21      0.06      0.10      1000
           3       0.30      0.09      0.14      1000
           4       0.32      0.13      0.18      1000
           5       0.32      0.11      0.17      1000
           6       0.45      0.22      0.29      1000
           7       0.47      0.22      0.30      1000
           8       0.38      0.25      0.31      1000
           9       0.43      0.24      0.31      1000

   micro avg       0.40      0.18      0.25     10000
   macro avg       0.38      0.18      0.24     10000
weighted avg       0.38      0.18      0.24     10000
 samples avg       0.15      0.18      0.16     10000



/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [354]:
#Save model 
filename = './checkpoints/case2/LogisticRegression_Cifar.sav'
pickle.dump(lr_one_rest, open(filename, 'wb'))

##### 2.2.2.2 Sub shadow model  : Random forest : 

In [18]:
# Now Random forests 
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(x_D_shadow_train, y_D_shadow_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [352]:
#Eval model 
pred5=random_forest.predict(x_test)
print(classification_report(y_test, pred5))

              precision    recall  f1-score   support

           0       0.79      0.22      0.35       258
           1       0.75      0.01      0.02       252
           2       0.00      0.00      0.00       236
           3       0.00      0.00      0.00       286
           4       0.50      0.01      0.02       221
           5       0.88      0.02      0.05       282
           6       0.71      0.02      0.04       244
           7       0.67      0.01      0.02       240
           8       0.78      0.22      0.34       233
           9       0.86      0.02      0.05       248

   micro avg       0.78      0.05      0.10      2500
   macro avg       0.59      0.05      0.09      2500
weighted avg       0.59      0.05      0.09      2500
 samples avg       0.05      0.05      0.05      2500



/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [355]:
#Save model 
filename = './checkpoints/case2/RandomForest_Cifar.sav'
pickle.dump(random_forest, open(filename, 'wb'))

##### 2.2.2.3 Stacking submodels : 

In [29]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

estimators = [
            ('lr', OneVsRestClassifier(LogisticRegression(random_state=42,
                                                          multi_class='multinomial',
                                                         solver='sag',max_iter=10))),
            ('rf',RandomForestClassifier(random_state=42))
    
            ]
clf2 = StackingClassifier(estimators=estimators)

In [ ]:
clf2.fit(x_D_shadow_train, y_D_shadow_train);

/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/hom

/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_ite

In [366]:
#Eval model 
pred6=clf2.predict(x_test)
print(classification_report(y_test, pred6))

              precision    recall  f1-score   support

           0       0.79      0.22      0.35       258
           1       0.75      0.01      0.02       252
           2       0.00      0.00      0.00       236
           3       0.00      0.00      0.00       286
           4       0.50      0.01      0.02       221
           5       0.88      0.02      0.05       282
           6       0.71      0.02      0.04       244
           7       0.67      0.01      0.02       240
           8       0.78      0.22      0.34       233
           9       0.86      0.02      0.05       248

   micro avg       0.78      0.05      0.10      2500
   macro avg       0.59      0.05      0.09      2500
weighted avg       0.59      0.05      0.09      2500
 samples avg       0.05      0.05      0.05      2500



/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/soumia/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [367]:
#Save model 
filename = './checkpoints/case2/stacked_models_Cifar.sav'
pickle.dump(clf2, open(filename, 'wb'))

##### 2.2.3 construct Attack data : 
Make sure to run the cell of [This function](#Something) Before continuing in this section

In [37]:
##### print(len(x_D_shadow_out))
probas = random_forest.predict_proba(x_D_shadow_out)
pipo = np.asarray(probas) 
print(pipo.shape)
maxos =  np.argmax(pipo, axis=2)
p = maxos.reshape(12500,10)

probs2 = lr_one_rest.predict_proba(x_D_shadow_out)
print(p.shape)
print(probs2[0])

(10, 12500, 2)
(12500, 10)
[0.02433537 0.0244615  0.28429316 0.21142529 0.08037525 0.28627002
 0.05598005 0.02341974 0.10610243 0.00353619]


In [36]:
print(maxos.shape)
print(probs2.shape)

(10, 12500)
(12500, 10)


In [9]:
# This cell has already been processed and data are saved 
# There is no need to re-run it again
attack_train_save = "data/case2/cifar10/attack_train_dataset.csv"
attack_test_save = "data/case2/cifar10/attack_test_dataset.csv"

df = construct_dataset(x1=x_D_shadow_train,
                       x2=x_D_shadow_out,
                       clf=clf2, 
                       path_save=attack_train_save)

df = construct_dataset(x1=x_D_target_train,
                       x2=x_D_target_eval,
                       clf=clf2, 
                       path_save=attack_test_save)

In [10]:
train_file_path='data/case2/cifar10/attack_train_dataset.csv'
test_file_path='data/case2/cifar10/attack_test_dataset.csv'
attack_loaders = loader.get_attack_train_test_loaders(dataset_train_path=train_file_path,
                                                      dataset_test_path=test_file_path,
                                                        batch_size=10)

##### 2.2.4 train attack model and evaluate it : 

In [11]:
attack_model = Attack_classifier(3)

In [12]:
train_dataloader = attack_loaders['train_loader']

In [14]:
optimizer = Adam(attack_model.parameters(), lr=0.000001)

train_scores= TU.train(train_dataloader, 
                    optimizer,
                    model=attack_model,
                    n_epochs=10, 
                    model_path="./checkpoints/case2/cifar_attack_model.pth")

Epoch 1 - Training loss: 0.6968972384214401
Epoch 2 - Training loss: 0.6964005449771881
Epoch 3 - Training loss: 0.697587501335144
Epoch 4 - Training loss: 0.6964075799226761
Epoch 5 - Training loss: 0.6963178188323974
Epoch 6 - Training loss: 0.6969622575044632
Epoch 7 - Training loss: 0.696983782529831
Epoch 8 - Training loss: 0.6966662009954453
Epoch 9 - Training loss: 0.6969615666627884
Epoch 10 - Training loss: 0.6962411234855652


In [15]:
acc_attack,precision_attack,recall_attack = TU.eval_model(attack_model,attack_loaders['test_loader'])


Average Val Loss: 0.0693, Val Accuracy: (0.498%) , precision:  0.4972, recall :  0.3442

